In [65]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
%matplotlib inline

In [66]:
fields = ['Sentence','Word', 'POS', 'Tag']

train = pd.read_csv("./data/TrainNER.csv",sep=';',na_values='', encoding='latin1')

test1 = pd.read_csv("./data/Test1NER.csv",sep=';',na_values='', encoding='latin1')
test2 = pd.read_csv("./data/Test2NER.csv",sep=';',na_values='', encoding='latin1')

#Add columns to test1 df
test1.loc[-1] = [test1.columns[0], test1.columns[1], test1.columns[2]]  # rename columns
test1.index = test1.index + 1  # shifting index
test1 = test1.sort_index()  # sorting by index
test1 = test1.rename(columns={test1.columns[0]: 'Sentences', test1.columns[1]: 'Word',test1.columns[2]:  'POS'}) # adding first row

#Add columns to test2 df
test2.loc[-1] = [test2.columns[0], test2.columns[1]]  # rename columns
test2.index = test2.index + 1  # shifting index
test2 = test2.sort_index()  # sorting by index
test2 = test2.rename(columns={test2.columns[0]: 'Sentences', test2.columns[1]: 'Word'}) # adding first row

test1_format = pd.read_csv("./data/Test1NER_PredictedFormat.csv",sep=';',na_values='', encoding='latin1')
test2_format = pd.read_csv("./data/Test2NER_PredictedFormat.csv",sep=';',na_values='', encoding='latin1')

#print(test1.iloc[0:2])

        Sentences  Word POS
0  Sentence: 9001    In  IN
1             NaN  2005  CD


In [67]:
#test1
test1_words_with_pos = list()
for w in range(len(test1.index)):
    if pd.notna(test1[w:w+1]['Word'].values[0]):
        tup = (test1[w:w+1]['Word'].values[0] , test1[w:w+1]['POS'].values[0])
        test1_words_with_pos.append(tup)
#print(test1_words_with_pos)


nts', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('Gaza', 'NNP'), ('Strip', 'NNP'), ('.', '.'), ('Hundreds', 'NNS'), ('of', 'IN'), ('Libyan', 'JJ'), ('opposition', 'NN'), ('members', 'NNS'), ('have', 'VBP'), ('concluded', 'VBN'), ('a', 'DT'), ('two-day', 'JJ'), ('conference', 'NN'), ('in', 'IN'), ('London', 'NNP'), ('to', 'TO'), ('discuss', 'VB'), ('strategies', 'NNS'), ('for', 'IN'), ('ousting', 'VBG'), ('Moammar', 'NNP'), ('Gadhafi', 'NNP'), ('from', 'IN'), ('power', 'NN'), ('.', '.'), ('The', 'DT'), ('opposition', 'NN'), ('leaders', 'NNS'), ('are', 'VBP'), ('calling', 'VBG'), ('on', 'IN'), ('the', 'DT'), ('United', 'NNP'), ('Nations', 'NNPS'), ('to', 'TO'), ('push', 'VB'), ('for', 'IN'), ('regime', 'NN'), ('change', 'NN'), ('in', 'IN'), ('Libya', 'NNP'), ('and', 'CC'), ('for', 'IN'), ('the', 'DT'), ('restoration', 'NN'), ('of', 'IN'), ('Libya', 'NNP'), ("'s", 'POS'), ('constitution', 'NN'), ('.', '.'), ('The', 'DT'), ('leaders', 'NNS'), ('said', 'VBD'), ('Sunday', 'NNP'), ('they', 'PRP'),

In [68]:

from nltk.tag import StanfordNERTagger
# Stanford NER tagger    
def stanford_tagger(token_text):
	st = StanfordNERTagger('./stanford-ner-4.0.0/classifiers/english.muc.7class.distsim.crf.ser.gz',
							'./stanford-ner-4.0.0/stanford-ner-4.0.0.jar',
							encoding='utf-8')   
	ne_tagged = st.tag(token_text)
	return(ne_tagged)

In [73]:
def bio_tagger(ne_tagged):
		bio_tagged = []
		prev_tag = "O"
		for token, tag in ne_tagged:
			if tag == "O": #O
				bio_tagged.append((token, tag))
				prev_tag = tag
				continue
			if tag != "O" and prev_tag == "O": # Begin NE
				bio_tagged.append((token, "B-"+tag))
				prev_tag = tag
			elif prev_tag != "O" and prev_tag == tag: # Inside NE
				bio_tagged.append((token, "I-"+tag))
				prev_tag = tag
			elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
				bio_tagged.append((token, "B-"+tag))
				prev_tag = tag
		return bio_tagged

In [74]:
#test1
only_words = list()
for w in test1['Word']:
    if pd.notna(w):
        only_words.append(w)
#print(only_words)

tokens,ne_tags = zip(*bio_tagger(stanford_tagger(only_words)))

test1["Predicted"] = ne_tags

#test2
only_words = list()
for w in test2['Word']:
    if pd.notna(w):
        only_words.append(w)
#print(only_words)

tokens,ne_tags = zip(*bio_tagger(stanford_tagger(only_words)))

test2["Predicted"] = ne_tags
#print(structure_ne(stanford_tree(bio_tagger(stanford_tagger(only_words)))))

In [71]:
#beküldéshez
for w in test1.index:
    if test1.loc[w,'Predicted']=='B-LOCATION':
        test1.loc[w,'Predicted']='B-geo'
    if test1.loc[w,'Predicted']=='I-LOCATION':
        test1.loc[w,'Predicted']='I-geo'

    if test1.loc[w,'Predicted']=='B-PERSON':
        test1.loc[w,'Predicted']='B-per'
    if test1.loc[w,'Predicted']=='I-PERSON':
        test1.loc[w,'Predicted']='I-per'

    if test1.loc[w,'Predicted']=='B-DATE':
        test1.loc[w,'Predicted']='O'
    if test1.loc[w,'Predicted']=='I-DATE':
        test1.loc[w,'Predicted']='O'

    if test1.loc[w,'Predicted']=='B-ORGANIZATION':
        test1.loc[w,'Predicted']='B-org'
    if test1.loc[w,'Predicted']=='I-ORGANIZATION':
        test1.loc[w,'Predicted']='I-org'

    if test1.loc[w,'Predicted']=='B-TIME':
        test1.loc[w,'Predicted']='B-time'
    if test1.loc[w,'Predicted']=='I-TIME':
        test1.loc[w,'Predicted']='I-time'

    if test1.loc[w,'Predicted']=='B-MONEY':
        test1.loc[w,'Predicted']='O'
    if test1.loc[w,'Predicted']=='I-MONEY':
        test1.loc[w,'Predicted']='O'

    if test1.loc[w,'Predicted']=='B-PERCENT':
        test1.loc[w,'Predicted']='O'
    if test1.loc[w,'Predicted']=='I-PERCENT':
        test1.loc[w,'Predicted']='O'


test1.to_csv("Submission_1.csv",index=False,sep = ';')

#beküldéshez
#submission2_df=test_df[['ID','p1']].copy()
#submission2_df.columns=['Id','Predicted']
#submission2_df.to_csv("JNH1DH.csv",index=False)

In [72]:
#beküldéshez
for w in test2.index:
    if test2.loc[w,'Predicted']=='B-LOCATION':
        test2.loc[w,'Predicted']='B-geo'
    if test2.loc[w,'Predicted']=='I-LOCATION':
        test2.loc[w,'Predicted']='I-geo'

    if test2.loc[w,'Predicted']=='B-PERSON':
        test2.loc[w,'Predicted']='B-per'
    if test2.loc[w,'Predicted']=='I-PERSON':
        test2.loc[w,'Predicted']='I-per'

    if test2.loc[w,'Predicted']=='B-DATE':
        test2.loc[w,'Predicted']='O'
    if test2.loc[w,'Predicted']=='I-DATE':
        test2.loc[w,'Predicted']='O'

    if test2.loc[w,'Predicted']=='B-ORGANIZATION':
        test2.loc[w,'Predicted']='B-org'
    if test2.loc[w,'Predicted']=='I-ORGANIZATION':
        test2.loc[w,'Predicted']='I-org'

    if test2.loc[w,'Predicted']=='B-TIME':
        test2.loc[w,'Predicted']='B-time'
    if test2.loc[w,'Predicted']=='I-TIME':
        test2.loc[w,'Predicted']='I-time'

    if test2.loc[w,'Predicted']=='B-MONEY':
        test2.loc[w,'Predicted']='O'
    if test2.loc[w,'Predicted']=='I-MONEY':
        test2.loc[w,'Predicted']='O'

    if test2.loc[w,'Predicted']=='B-PERCENT':
        test2.loc[w,'Predicted']='O'
    if test2.loc[w,'Predicted']=='I-PERCENT':
        test2.loc[w,'Predicted']='O'


test2.to_csv("Submission_2.csv",index=False,sep = ';')

#beküldéshez
#submission2_df=test_df[['ID','p1']].copy()
#submission2_df.columns=['Id','Predicted']
#submission2_df.to_csv("JNH1DH.csv",index=False)